# Snakemake Manual

### Snakefile

Snakemake 버젼을 정해주고 Config 파일들을 읽을 수 있게 지정해준다

In [ ]:
from snakemake.utils import min_version
import pandas as pd

##### set minimum snakemake version #####
min_version("6.4.1")

configfile: "workflow/config.yaml"
configfile: "workflow/tools.yaml"
configfile: "workflow/params.yaml"

common.smk를 꼭 include하도록 포함시켜준다.<br>
common.smk는 Snakemake의 기본 설정과 function들이 담겨있는 파일로, 이후에 자세한 설명을 하겠다.

In [ ]:
include: "rules/common.smk"

Configuration 파일들의 설정에 따라, 어떤 smk 파일들을 포함시킬지 설정을 해준다.

In [ ]:
if config['qc'] == 'fastqc':
    include: "rules/fastqc.smk"

if config['align'] == 'star':
    include: "rules/starindex.smk"
    include: "rules/star.smk"
    include: "rules/sam.smk"

if config['trim'] == 'trimmomatic' and config['ends'] == 'PE':
    include: "rules/trimPE.smk"
elif config['trim'] == 'trimmomatic' and config['ends'] == 'SE':
    include: "rules/trimSE.smk"

if config['trim'] == 'trimgalore' and config['ends'] == 'PE':
    include: "rules/galorePE.smk"
elif config['trim'] == 'trimgalore' and config['ends'] == 'SE':
    include: "rules/galoreSE.smk"

if config['quant'] == 'rsem':
    include: "rules/rsemindex.smk"
    include: "rules/rsem.smk"
elif config['quant'] == 'htseq':
    include: "rules/htseq.smk"

추가하고 싶은 툴이 있다면 여기에 꼭 포함을 시킨다. 예를 들어 aligner BWA를 추가하고 싶으면 

In [ ]:
if config['align'] == 'star':
    include: "rules/starindex.smk"
    include: "rules/star.smk"
    include: "rules/sam.smk"
elif config['align'] == 'bwa':
    include: "rules/bwa.smk"

와 같이 if문을 사용해서 smk 파일을 지정해준다.
아래와 같이 tools.yaml의 설정도 수정해줘야한다.

수정 전:

In [ ]:
# alignment tool <star>
align: star

수정 후:

In [ ]:
# alignment tool <star/bwa>
# align: star
align: bwa

Snakefile의 마지막으로, rule all은 최종적으로 output이 될 파일들을 모두 list 해준다. <br>
실제 list는 common.smk에서 all_input라는 function에 지정해준다.

In [ ]:
rule all:
    input:
        all_input,

### common.smk

In [ ]:
from snakemake.utils import validate
import pandas as pd
import numpy as np

samples.csv와 genome.csv를 읽어준다. <br>
대문자 변수들은 각 csv 파일의 정보를 담은 list라고 볼 수 있다. <br>
EXTOP은 fastq 파일의 형식이 여러가지 있을 수도 있어 그 경우의 수를 list로 지정해준 것이다.

In [ ]:

df = pd.read_csv(config['samples'], dtype='str').set_index(
    ["sample_name", "reads", "fq", "ext"], drop=False)

gdf = pd.read_csv(config['genome'], dtype='str').set_index(
    ["name", "fa", "annot"], drop=False)

SAMPLES = df.sample_name
READS = df.reads
FULL = df.fq
EXT = df.ext
FASTA = gdf.fa
GTF = gdf.annot
NAME = gdf.name
EXTOP = ['fastq.gz', 'fq.gz', 'fastq', 'fq']

In [ ]:
wildcard_constraints:
    sample = "|".join(df.sample_name),
    read = "|".join(df.reads),
    full = "|".join(df.fq),
    fasta = "|".join(gdf.fa),
    ann = "|".join(gdf.annot),
    ext = "|".join(df.ext),

In [ ]:
def is_single_end(sample, read):
    """Determine whether single-end."""
    fq2_not_present = df.loc[pd.isnull(df['reads'])]
    return fq2_not_present["sample_name"]

In [ ]:
def get_fastqs(wildcards):
    """Get raw FASTQ files from unit sheet."""
    if config['ends'] == 'SE':
        return [f'data/{df.loc[(wildcards.sample), "fq"]}']
    else:
        for i in range(0, len(df)):
            u = df.loc[(wildcards.sample), ["fq"]].dropna()
            return [f'data/{u.fq[i]}', f'data/{u.fq[i + 1]}']

In [ ]:
def get_trimmed(wildcards):
    """Get raw FASTQ files from unit sheet."""
    if config['ends'] == 'SE':
        return [f'trimmed/{config["trim"]}/{df.loc[(wildcards.sample), "fq"]}']
    else:
        for i in range(0, len(df)):
            u = df.loc[(wildcards.sample), ["fq"]].dropna()
            return [f'trimmed/{config["trim"]}/{u.fq[i]}', f'trimmed/{config["trim"]}/{u.fq[i + 1]}']



all_input function은 Snakemake가 최종 output file의 wildcard를 알 수 있게 해주는 부분이다. <br>
여기서 output file들의 wildcard를 지정해주면 Snakemake에서 그것을 기반으로 input file들의 이름들을 자동으로 읽어준다.

In [ ]:
def all_input(wildcards):
    """
    Function defining all requested inputs for the rule all (below).
    """
    wanted_input = []

    wanted_input.extend(
            [directory("./genome/")]
        )

    if config['qc']=='fastqc':
        wanted_input.extend(
            expand(
                ["qc/{tool}/{id.sample_name}_{id.reads}_fastqc.html",
                "qc/{tool}/{id.sample_name}_{id.reads}_fastqc.zip"], 
                id=df[['sample_name', 'reads']].itertuples(), tool= config['qc']
            )
        )

    if config['trim']=='trimmomatic' and config['ends'] == 'PE':
        wanted_input.extend(
            expand(
                ["trimmed/{tool}/{id.sample_name}_{id.reads}.{id.ext}",
                "trimmed/{tool}/{id.sample_name}_{id.reads}.se.{id.ext}"], 
                id=df[['sample_name', 'reads', 'ext']].itertuples(), tool= config['trim']
            )
        )
    elif config['trim']=='trimmomatic' and config['ends'] == 'SE':
        wanted_input.extend(
            expand(
                ["trimmed/{tool}/{id.sample_name}.{id.ext}"], 
                id=df[['sample_name', 'ext']].itertuples(), tool= config['trim']
            )
        )

    if config['trim']=='trimgalore' and config['ends'] == 'PE':
        wanted_input.extend(
            expand(
                ["trimmed/{tool}/{id.sample_name}_{id.reads}.{id.ext}",
                "trimmed/{tool}/{id.sample_name}_{id.reads}.{id.ext}.report.txt"], 
                id=df[['sample_name', 'reads', 'ext']].itertuples(), tool= config['trim']
            )
        )
    elif config['trim']=='trimgalore' and config['ends'] == 'SE':
        wanted_input.extend(
            expand(
                ["trimmed/{tool}/{id.sample_name}.{id.ext}",
                "trimmed/{tool}/{id.sample_name}.{id.ext}.report.txt"], 
                id=df[['sample_name', 'ext']].itertuples(), tool= config['trim']
            )
        )

    if config['align']=='star':
        wanted_input.extend(
            [directory("genome/starindex/")]
        )
    
    if config['align']=='star' and config['ends'] == 'PE':
        wanted_input.extend(
            expand(
                ["aligned/{tool}/pe/{id.sample_name}.Aligned.sortedByCoord.out.bam",
                "aligned/{tool}/pe/{id.sample_name}.Aligned.toTranscriptome.out.bam",
                "aligned/{tool}/pe/{id.sample_name}.Log.out",
                "aligned/{tool}/pe/{id.sample_name}.SJ.out.tab",
                "aligned/{tool}/{id.sample_name}.Aligned.toTranscriptome.sorted.bam",],
                id=df[['sample_name']].itertuples(), tool= config['align']
            )
        )
    elif config['align']=='star' and config['ends'] == 'SE':
        wanted_input.extend(
            expand(
                ["aligned/{tool}/se/{id.sample_name}.Aligned.sortedByCoord.out.bam",
                "aligned/{tool}/se/{id.sample_name}.Aligned.toTranscriptome.out.bam",
                "aligned/{tool}/se/{id.sample_name}.Log.out",
                "aligned/{tool}/se/{id.sample_name}.SJ.out.tab",
                "aligned/{tool}/{id.sample_name}.Aligned.toTranscriptome.sorted.bam",],
                id=df[['sample_name']].itertuples(), tool= config['align']
            )
        )

    if config['quant']=='rsem':
        wanted_input.extend(
            expand(
                ["genome/rsemindex/{a.name}.seq",
                "quant/{tool}/{id.sample_name}.genes.results",
                "quant/{tool}/{id.sample_name}.isoforms.results"],
                a=gdf[['name']].itertuples(), id=df[['sample_name']].itertuples(), tool=config['quant']
            )
        )

    if config['quant']=='htseq':
            wanted_input.extend(
                expand(
                    ["quant/{tool}/{id.sample_name}_count.tsv"],
                    id=df[['sample_name']].itertuples(), tool= config['quant']
                )
            )

    return wanted_input

예시로 fastqc를 살펴보겠다.

In [ ]:
if config['qc']=='fastqc':
    wanted_input.extend(
        expand(
            ["qc/{tool}/{id.sample_name}_{id.reads}_fastqc.html",
            "qc/{tool}/{id.sample_name}_{id.reads}_fastqc.zip"], 
            id=df[['sample_name', 'reads']].itertuples(), tool= config['qc']
        )
    )

tools.yaml 파일에서 qc가 'fastqc'로 지정되어있을 때, 'wanted_input'이라는 변수에 파일명을 추가해준다.

In [ ]:
    expand(
        ["qc/{tool}/{id.sample_name}_{id.reads}_fastqc.html",
        "qc/{tool}/{id.sample_name}_{id.reads}_fastqc.zip"], 
        id=df[['sample_name', 'reads']].itertuples(), tool= config['qc']
    )

이 부분이 wildcard들을 지정해주는데, samples.csv에서 가져온 정보와 tools.yaml의 설정을 이용해서 가능한 파일명의 조합을 만들어준다. <br>
tools.yaml에서 qc를 'fastqc'라고 설정이 되어있는 경우, <br>
sample_name이 "control_1", "control_2", "case_1", "case_2"가 있고, <br>
reads는 "1", "2"가 있다고 하면,

qc/fastqc/control_1_1_fastqc.html<br>
qc/fastqc/control_1_2_fastqc.html<br>
qc/fastqc/control_2_1_fastqc.html<br>
qc/fastqc/control_2_2_fastqc.html<br>
qc/fastqc/case_1_1_fastqc.html<br>
qc/fastqc/case_1_2_fastqc.html<br>
qc/fastqc/case_2_1_fastqc.html<br>
qc/fastqc/case_2_2_fastqc.html<br>


output 파일들이 만들어질 것이라고 Snakemake에서 인식을 한다.

그렇다면, fastqc.smk를 살펴보자.

In [ ]:
rule fastqc:
    input:
        expand("data/{{sample}}_{{read}}.{ext}", ext=EXT[0])
    output:
        html = "qc/{qctool}/{sample}_{read}_fastqc.html",
        zip = "qc/{qctool}/{sample}_{read}_fastqc.zip"
    log:
        "qc/{qctool}/logs/{sample}_{read}.fastqc.log"
    threads: config['threads']
    wrapper:
         "https://raw.githubusercontent.com/bmi-rna-pipeline/snakemake-wrappers/master/bio/fastqc"

여기서도 wildcard가 나오지만, 직접 samples.csv에서 정보를 가져오지 않는다. <br>
fastqc.smk 파일의 wildcard들은 common.smk에서 알려준 wildcard로 추정을 한다. 

In [ ]:
    output:
        html = "qc/{qctool}/{sample}_{read}_fastqc.html",
        zip = "qc/{qctool}/{sample}_{read}_fastqc.zip"

output 파일들을 봤을 때, 

common.smk에서 알려준대로의 똑같은 형식으로 되어있기 때문에, Snakemake에서 알아서 인식을 해준다.

In [ ]:
    expand(
        ["qc/{tool}/{id.sample_name}_{id.reads}_fastqc.html",
        "qc/{tool}/{id.sample_name}_{id.reads}_fastqc.zip"], 
        id=df[['sample_name', 'reads']].itertuples(), tool= config['qc']
    )

하지만, common.smk에서 알려주지 않은 부분을 wildcard로 넣고 싶을 때가 있을 것이다. <br>
input 파일에서 fastq 파일의 확장자를 wildcard로 지정하고 싶은데, common.smk에서 알려준 wildcard에 그런 정보가 없다.

그런 경우, common.smk파일에서 만들어준 리스트를 활용한다. (이것을 매번 사용하지 않는 이유는 코드가 불안정해지기 때문이다.)

In [ ]:
df = pd.read_csv(config['samples'], dtype='str').set_index(
    ["sample_name", "reads", "fq", "ext"], drop=False)

gdf = pd.read_csv(config['genome'], dtype='str').set_index(
    ["name", "fa", "annot"], drop=False)

SAMPLES = df.sample_name
READS = df.reads
FULL = df.fq
EXT = df.ext
FASTA = gdf.fa
GTF = gdf.annot
NAME = gdf.name
EXTOP = ['fastq.gz', 'fq.gz', 'fastq', 'fq']

samples.csv의 첫번째 파일의 확장자를 가지고 오겠다. EXT에 저장을 해놓았기 때문에, EXT[0]을 wildcard로 만들어준다. 

In [ ]:
    input:
        expand("data/{{sample}}_{{read}}.{ext}", ext=EXT[0])

expand()는 wildcard의 모든 가능한 조합을 가지고 리스트를 만들어주는 것이다. <br>
하지만 sample과 read는 반복해서 expand하려는 변수가 아니라, output파일에서 이미 알고 있는 변수이기 때문에 괄호를 하나 더 씌워준다.

EXT[0] = 'fq.gz'일 경우,

In [ ]:
    input:
        expand("data/{{sample}}_{{read}}.{ext}", ext=EXT[0])

는

In [ ]:
    input:
        "data/{sample}_{read}.fq.gz"

와 최종적으로 같은 기능을 해주는 것이다.

만약

In [ ]:
EXT = ["fq.gz", "fq.gz", "fq.gz", "fq.gz"]

일 때,

In [ ]:
    input:
        expand("data/{{sample}}_{{read}}.{ext}", ext=EXT)

와 같이 wildcard를 EXT 전체로 지정해주면, snakemake는 모든 조합을 사용하기 때문에,

In [ ]:
    input:
        "data/{sample}_{read}.fq.gz"

이 아닌,

In [ ]:
    input:
        "data/{sample}_{read}.fq.gz",
        "data/{sample}_{read}.fq.gz",
        "data/{sample}_{read}.fq.gz",
        "data/{sample}_{read}.fq.gz"

이런 형태로 인식을 할 수 있기 때문에 EXT[0]하나만 지정한 것이다.

## Tutorial

새로운 툴을 snakemake에 추가해보자.

우선, smk파일을 만들어준다.

예시로, trimmomatic Paired end 사용을 위한 smk를 만들어보자.

먼저, output 파일의 형태를 알고 있어야한다.

In [ ]:
    output:
        r1 = "trimmed/{trtool}/{sample}_1.{ext}",
        r2 = "trimmed/{trtool}/{sample}_2.{ext}",
        r1_unpaired = "trimmed/{trtool}/{sample}_1.se.{ext}",
        r2_unpaired = "trimmed/{trtool}/{sample}_2.se.{ext}"

In [ ]:
    input:
        r1 = "data/{sample}_1.{ext}",
        r2 = "data/{sample}_2.{ext}"